# Load dataset and visualize

In [2]:
with open('shakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(f"Length of text: {len(text)}")

Length of text: 1115389


In [5]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
chars = sorted(list(set(text)))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65


# Tokenize text

In [9]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
# encoder
encode = lambda s: [stoi[c] for c in s]
# decoder
decode = lambda l: ''.join([itos[i] for i in l])

In [12]:
print(encode("hi there"))
print(decode(encode("hi there")))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [18]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115389]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [19]:
# split into train and validation
n = int(0.9 * len(data))
train_data = data[:n]
valid_data = data[n:]

In [20]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [21]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Input {context} -> Output {target}")

Input tensor([18]) -> Output 47
Input tensor([18, 47]) -> Output 56
Input tensor([18, 47, 56]) -> Output 57
Input tensor([18, 47, 56, 57]) -> Output 58
Input tensor([18, 47, 56, 57, 58]) -> Output 1
Input tensor([18, 47, 56, 57, 58,  1]) -> Output 15
Input tensor([18, 47, 56, 57, 58,  1, 15]) -> Output 47
Input tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> Output 58


In [23]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size, size=(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size + 1] for i in ix])
    return x, y

x, y = get_batch('train')
print(x.shape)
print(x)
print(y.shape)
print(y)

torch.Size([4, 8])
tensor([[59, 57,  1, 58, 56, 39, 47, 58],
        [30, 10,  0, 15, 53, 51, 43,  6],
        [34, 21, 26, 15, 17, 26, 32, 21],
        [27, 10,  0, 32, 46, 53, 59,  1]])
torch.Size([4, 8])
tensor([[57,  1, 58, 56, 39, 47, 58, 53],
        [10,  0, 15, 53, 51, 43,  6,  1],
        [21, 26, 15, 17, 26, 32, 21, 27],
        [10,  0, 32, 46, 53, 59,  1, 42]])
